# Using Row Access Policies with Cortex Analyst
This Notebook will help setup a Row Access Poicy (RAP) on the Cortex Analyst `DAILY_REVENUE` table to illustrate how we can restrict a tenant's access to data in Snowflake in a way that allows us to use the SQL generated by Cortex Analyst.

The challenge with Cortex Analyst is that the SQL result from Cortex Analyst is a SQL statement in a string. If we have a multi-tenant scenario with data from the various tenants mixed together in a multi-tenant table (a common design pattern), then we need to modify the SQL statement to ensure that each tenant can only see their data and not data from other tenants. Since the SQL statement from Cortex Analyst is a string, we would need to parse the string in order to determine where to place the predicates to restrict the data access to a given tenant. This is rather challenging.

As an alternative, we can put the data restictions into a Row Access Policy on the tables, which will ensure that the query will only access the data for a given tenant. In order to do this, we need some context in the Snowflake session that indicates which tenant is the "active tenant". Then, the Row Access Policy can use that session context to restrict the data acess to the specified tenant. A common pattern is to use a helper entitlements table that will map the session context value to the allowed values in the data tables.

This Notebook shows how we can use a session variable approach to set the context for the tenant. First, we create an entitlement table that maps each tenant to a value in our data tables. For this example, we are going to restrict tenant 1 (represented by key `RL_TENANT_1`) to data from region `1`, tenant 2 to region `2`, and so on. We will use the session variable `TENANT` to hold the value of the tenant of interest. Lastly, we will create a Row Access Policy using the `GETVARIABLE('TENANT')` function and the entitlement table to restrict the data based on the `region_id` column.

Then, our application will connect to Snowflake using a service user that has the application role. When it wants to query as tenant 1, it will first set the `TENANT` variable to `RL_TENANT_1` (using a command like `SET TENANT = 'RL_TENANT_1'`), and then issue the query. 

## 1. Create the necessary roles
First we create the application role and the roles per tenant, and we grant each tenant role to the applicaiton role. We also grant the application role to the current user.

In [ ]:
USE ROLE securityadmin;
CREATE ROLE IF NOT EXISTS rl_app;

SET cur_user = CURRENT_USER();
GRANT ROLE rl_app TO USER IDENTIFIER($cur_user);

## 2. Create the Entitlements Table
Next we will create the entitlements table that will map which roles have access to which region IDs. Additionally, we will allow the `CORTEX_USER_ROLE` to have access to all of the data, so we add a row in the entitlement table for each region ID.

In [ ]:
USE ROLE cortex_user_role;
USE SCHEMA cortex_analyst_demo.revenue_timeseries;

DROP TABLE IF EXISTS daily_rev_ent_region;
CREATE TABLE daily_rev_ent_region(key STRING, value INT);

-- Let's make a key that can access all of the data
INSERT INTO daily_rev_ent_region
    SELECT 'ALL' AS key, region_id AS value 
    FROM daily_revenue 
    GROUP BY region_id
;

-- Now, let's restrict tenants to just their region
INSERT INTO daily_rev_ent_region(key,value) VALUES 
    ('RL_TENANT_1', 1), 
    ('RL_TENANT_2', 2), 
    ('RL_TENANT_3', 3), 
    ('RL_TENANT_4', 4), 
    ('RL_TENANT_5', 5)
;

SELECT * FROM daily_rev_ent_region ORDER BY key, value;

## 3. Set up the Row Access Policy
In order to make the Row Access Policy performant, we will create a memoizable function that collects all `VALUE`s that a given `KEY` has access to. Then, we create a Row Access Policy that uses that function, passing in the `CURRENT_ROLE()` as the argument. Since this is contant for the length of the query, the UDF will run once and the result reused (as is the behavior of memoizable functions).

In [ ]:
USE ROLE cortex_user_role;
USE SCHEMA cortex_analyst_demo.revenue_timeseries;

DROP FUNCTION IF EXISTS udf_regions(STRING);
CREATE FUNCTION udf_regions(ikey STRING)
    RETURNS ARRAY
    MEMOIZABLE
    AS 'SELECT ARRAY_AGG(value) FROM daily_rev_ent_region WHERE key = ikey';

DROP ROW ACCESS POLICY IF EXISTS rap_region_var;
CREATE ROW ACCESS POLICY rap_region_var
    AS (region_id INT)
    RETURNS BOOLEAN ->
        CASE 
            WHEN 'RL_APP' = CURRENT_ROLE() THEN  ARRAY_CONTAINS(region_id, udf_regions(GETVARIABLE('TENANT')))
            ELSE TRUE
        END        
;

Now we add this Row Access Policy to the `REGION_ID` column of the `DAILY_REVENUE` table.

In [ ]:
ALTER TABLE daily_revenue DROP ALL ROW ACCESS POLICIES; -- Just to make sure there aren't any others
ALTER TABLE daily_revenue ADD ROW ACCESS POLICY rap_region_var ON (region_id);

## 4. Testing
Now we can test the access to the `DAILY_REVENUE` table. First, let's use the `CORTEX_ANALYST_ROLE` and ensure that all data is accessible.

In [ ]:
USE ROLE cortex_user_role;
SELECT region_id, COUNT(*) FROM daily_revenue GROUP BY region_id;

Next, let's use the `TENANT` variable to `RL_TENANT_1` to see that only data from region `1` is accessible.

In [ ]:
USE ROLE rl_app;
SET TENANT = 'RL_TENANT_1';
SELECT region_id, COUNT(*) FROM daily_revenue GROUP BY region_id;

Feel free to change the value of `TENANT` from `RL_TENANT_1` to `RL_TENANT_2` (to see data from region `2`), `RL_TENANT_3` (to see data from region `3`), `RL_TENANT_4` (to see data from region `4`), or `RL_TENANT_5` (to see data from region `5`).

## 5. Application
At this point, we can turn our attention to an application that will connect to Snowflake and use the various tenant roles to access the data as the various tenants. See the next steps in the GitHub repo for a sample application.

In [ ]:
USE ROLE rl_app;
SET TENANT = 'RL_TENANT_1';

WITH __daily_revenue AS (
  SELECT
    date,
    revenue AS daily_revenue
  FROM cortex_analyst_demo.revenue_timeseries.daily_revenue
)
SELECT
  SUM(daily_revenue) AS total_revenue
FROM __daily_revenue
WHERE
  date BETWEEN '2022-12-01' AND '2022-12-31'
 -- Generated by Cortex Analyst
;